In [32]:
import numpy as np
import pandas as pd
from functools import partial
from itertools import chain, combinations, zip_longest, groupby
from collections import Counter

In [2]:
ens_main = pd.read_csv('data/ens_main_int.csv', dtype={'chromosome_or_scaffold':'O', '5utr_start': 'Int64', '5utr_end': 'Int64', '3utr_start': 'Int64', '3utr_end': 'Int64', 'cds_start': 'Int64', 'cds_end': 'Int64'})
ens_exons = pd.read_csv('data/ens_exons_int.csv', dtype={'5utr_start': 'Int64', '5utr_end': 'Int64', '3utr_start': 'Int64', '3utr_end': 'Int64', 'cds_start': 'Int64', 'cds_end': 'Int64'})

In [3]:
ens_main = ens_main[ens_main['chromosome_or_scaffold'].isin([str(i) for i in list(range(1,23))] + ['X', 'Y'])]
ens_main = ens_main[ens_main['gene_type'] == 'protein_coding']
ens_main = ens_main[~ens_main['ensembl_gene_name'].isna()]

In [4]:
gene_size = ens_main.groupby('ensembl_gene_name').size()
gene_size = gene_size[gene_size > 1].index.to_list()
ens_main = ens_main[ens_main['ensembl_gene_name'].isin(gene_size)]

In [5]:
ens_min = ens_main[['ensembl_gene_name','gene_start', 'gene_end', 'ensembl_trs_id', 'trs_type', 'cds_start', 'cds_end']]

In [6]:
genes = ens_min['ensembl_gene_name'].unique()
len(genes)

16846

In [8]:
v = ens_min[ens_min['ensembl_gene_name'] == 'TEF']

In [15]:
v = ens_min[ens_min['ensembl_gene_name'] == 'TEF']
cdss = v[['cds_start', 'cds_end']].to_numpy()
cds_unique = np.sort(pd.unique(cdss.ravel()))

cds_arr = (cds_unique == cdss[..., None]).any(1).astype(int)

cds_seq = np.column_stack(list(zip_longest(*ens_exons[ens_exons['ensembl_gene_name'] == 'TEF']
           .groupby('ensembl_trs_id')[['exon_start', 'exon_end']]
           .apply(lambda x: [np.arange(a, b+1) for a, b in zip(x['exon_start'], x['exon_end'])])
           .apply(np.hstack).values, fillvalue=0)))

cds_seq_sum = (cds_unique == cds_seq[..., None]).any(1).astype(int).sum(0)

In [42]:
%%timeit
gene = 'TEF'

gene_sub = ens_min[ens_min['ensembl_gene_name'] == gene][['ensembl_trs_id', 'trs_type', 'cds_start', 'cds_end']]
gene_sub_pc = gene_sub[gene_sub['trs_type'] == 'protein_coding']
gene_exon_sub = ens_exons[ens_exons['ensembl_trs_id'].isin(gene_sub['ensembl_trs_id'])]

cds_ends = gene_sub[['cds_start', 'cds_end']].to_numpy()

cds_unique = np.sort(pd.unique(cds_ends.ravel()))

cds_mtx = (cds_unique == cds_ends[..., None]).any(1).astype(int)

seq_mtx = np.array((len(gene_sub), len(cds_unique)))

85.4 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
%%timeit
gene = 'TEF'

gene_sub = ens_min[ens_min['ensembl_gene_name'] == gene][['ensembl_trs_id', 'trs_type', 'cds_start', 'cds_end']]
gene_sub_pc = gene_sub[gene_sub['trs_type'] == 'protein_coding']
gene_exon_sub = ens_exons[ens_exons['ensembl_trs_id'].isin(gene_sub['ensembl_trs_id'])]

cds_ends = gene_sub[['cds_start', 'cds_end']].to_numpy()

cds_unique = np.sort(pd.unique(cds_ends.ravel()))

seq_mtx = np.zeros((len(gene_sub), len(cds_unique)))
seq_one = None

for i, trs in enumerate(gene_sub['ensembl_trs_id'].to_list()):
    gene_exon_trs = gene_exon_sub[gene_exon_sub['ensembl_trs_id'] == trs]
    seq_one = np.hstack([np.arange(a, b+1) for a, b in zip(gene_exon_trs['exon_start'], gene_exon_trs['exon_end'])])
    seq_mtx[i,:] = np.isin(cds_unique, seq_one)
    
seq_mtx = seq_mtx.astype(int)

118 ms ± 173 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [54]:
trs_prot_dup = []

gene = 'TEF'
gene_sub = ens_min[(ens_min['ensembl_gene_name'] == gene) & (ens_min['trs_type'] == 'protein_coding')][['ensembl_trs_id', 'trs_type', 'cds_start', 'cds_end']]
gene_start = 

In [43]:
%%timeit
cds_seq = np.column_stack(list(zip_longest(*ens_exons[ens_exons['ensembl_gene_name'] == 'TEF']
           .groupby('ensembl_trs_id')[['exon_start', 'exon_end']]
           .apply(lambda x: [np.arange(a, b+1) for a, b in zip(x['exon_start'], x['exon_end'])])
           .apply(np.hstack).values, fillvalue=0)))

89.8 ms ± 272 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
%%timeit

array([3, 5])

In [ ]:
e = ens_exon[ae3['ensembl_trs_id'] == j].loc[:, ['exon_genomic_start', 'exon_genomic_end']].copy()
exon_ranges = [np.arange(i[0], i[1]) for i in zip((exon_view.loc[:, "exon_genomic_start"] - gene_start).to_list(), (exon_view.loc[:, "exon_genomic_end"] - gene_start + 1).to_list())]
np.hstack(exon_ranges)

In [121]:
np.where((cds_arr == 1).any(1))

(array([0, 1, 2], dtype=int64),)

In [34]:
v['cds_start']

45914    41367533
45937    41382045
45942    41382935
Name: cds_start, dtype: Int64

In [50]:
np.stack([[0,1,2], [2,1,0]])

array([[0, 1, 2],
       [2, 1, 0]])

In [73]:
def rowin(u, c):
    return np.isin(u, c).astype(int)

UsageError: Line magic function `%%timeit` not found.


In [74]:
%%timeit
rowinl = partial(rowin, cds_unique)

153 ns ± 0.162 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [140]:
np.isin(np.where(cds_mtx==1, cds_mtx - cds_mtx.sum(0), 1)

array([[ 0,  1,  1,  1, -1],
       [ 1,  0,  1,  1, -1],
       [ 1,  1,  0,  0,  1]])

In [125]:
cds_unique

array([41367533, 41382045, 41382935, 41394274, 41395960], dtype=object)

In [115]:
(cds_unique == cdss[..., None]).any(1).astype(int)

array([[1, 0, 0, 0, 1],
       [0, 1, 0, 0, 1],
       [0, 0, 1, 1, 0]])

In [109]:
cdss[..., None]

array([[[41367533],
        [41395960]],

       [[41382045],
        [41395960]],

       [[41382935],
        [41394274]]], dtype=object)

In [110]:
cds_unique

array([41367533, 41382045, 41382935, 41394274, 41395960], dtype=object)

In [111]:
cds_unique == cdss[..., None]

array([[[ True, False, False, False, False],
        [False, False, False, False,  True]],

       [[False,  True, False, False, False],
        [False, False, False, False,  True]],

       [[False, False,  True, False, False],
        [False, False, False,  True, False]]])

In [ ]:
lambda x: np.hstack([np.arange(a, b+1) for a, b in zip(x['exon_start'], x['exon_end'])])

In [212]:
cds_seq = np.column_stack(list(zip_longest(*ens_exons[ens_exons['ensembl_gene_name'] == 'TEF']
           .groupby('ensembl_trs_id')[['exon_start', 'exon_end']]
           .apply(lambda x: [np.arange(a, b+1) for a, b in zip(x['exon_start'], x['exon_end'])])
           .apply(np.hstack).values, fillvalue=0)))

cds_seq_sum = (cds_unique == cds_seq[..., None]).any(1).astype(int).sum(0)

In [215]:
(cds_unique == cds_seq[..., None]).any(1).astype(int).sum(0)

array([1, 1, 1, 3, 2])

In [188]:
(cds_unique[..., None] == cds_seq).any(1).astype(int)

C:\Users\Dynev\AppData\Local\Temp\ipykernel_13732\1896111188.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  (cds_unique[..., None] == cds_seq).any(1).astype(int)


AttributeError: 'bool' object has no attribute 'any'

In [193]:
np.isin(cds_seq, cds_unique)

array([False, False, False])

In [195]:
(cds_unique == cds_seq[..., None]).any(1).astype(int)

C:\Users\Dynev\AppData\Local\Temp\ipykernel_13732\2603883795.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  (cds_unique == cds_seq[..., None]).any(1).astype(int)


AttributeError: 'bool' object has no attribute 'any'

In [134]:
ens_exons[ens_exons['exon_start'] == ens_exons['exon_end']]

,ensembl_trs_id,ensembl_exon_id,5utr_start,5utr_end,3utr_start,3utr_end,exon_start,exon_end,cds_start,cds_end
36308,ENST00000617794,ENSE00003745889,<NA>,<NA>,<NA>,<NA>,195798554,195798554,195798554,195798554
94418,ENST00000655539,ENSE00003859038,<NA>,<NA>,<NA>,<NA>,78787479,78787479,<NA>,<NA>
128296,ENST00000668966,ENSE00003871022,<NA>,<NA>,<NA>,<NA>,105707837,105707837,<NA>,<NA>
151289,ENST00000655121,ENSE00003874120,<NA>,<NA>,<NA>,<NA>,56137306,56137306,<NA>,<NA>
171364,ENST00000658440,ENSE00003858299,<NA>,<NA>,<NA>,<NA>,4243823,4243823,<NA>,<NA>
184313,ENST00000664259,ENSE00003863974,<NA>,<NA>,<NA>,<NA>,19753084,19753084,<NA>,<NA>
221838,ENST00000637577,ENSE00003832688,<NA>,<NA>,<NA>,<NA>,131795809,131795809,<NA>,<NA>
281327,ENST00000439769,ENSE00003831741,<NA>,<NA>,43540934,43540934,43540934,43540934,<NA>,<NA>
286564,ENST00000658725,ENSE00003888255,<NA>,<NA>,<NA>,<NA>,5446045,5446045,<NA>,<NA>
286653,ENST00000654287,ENSE00003888255,<NA>,<NA>,<NA>,<NA>,5446045,5446045,<NA>,<NA>


In [59]:
gene_sub

,ensembl_trs_id,trs_type,cds_start,cds_end
45914,ENST00000406644,protein_coding,41367533,41395960
45937,ENST00000266304,protein_coding,41382045,41395960
45942,ENST00000413942,protein_coding,41382935,41394274


In [82]:
cds_sub = ens_exons[ens_exons['ensembl_trs_id'] == 'ENST00000484142'].iloc[:, -2:].dropna().to_numpy()

In [83]:
np.sort(cds_sub.ravel())

array([63842186, 63842204, 63856306, 63856433, 63857829, 63857863],
      dtype=object)

In [79]:
ens_exons[ens_exons['ensembl_trs_id'] == 'ENST00000484142']

,ensembl_gene_name,ensembl_trs_id,ensembl_exon_id,5utr_start,5utr_end,3utr_start,3utr_end,exon_start,exon_end,cds_start,cds_end
1194278,UGP2,ENST00000484142,ENSE00001873492,63840954,63841232,<NA>,<NA>,63840954,63841232,<NA>,<NA>
1194279,UGP2,ENST00000484142,ENSE00001907097,63842059,63842185,<NA>,<NA>,63842059,63842204,63842186,63842204
1194280,UGP2,ENST00000484142,ENSE00003645549,<NA>,<NA>,<NA>,<NA>,63856306,63856433,63856306,63856433
1194281,UGP2,ENST00000484142,ENSE00003905523,<NA>,<NA>,<NA>,<NA>,63857829,63857863,63857829,63857863


In [76]:
def dup(gene):
    v = ens_min[(ens_min['ensembl_gene_name'] == gene) & (ens_min['trs_type'] == 'protein_coding')]['ensembl_trs_id'].values
    cdsrg = []
    
    for trs in v:
        cdsrg.append(np.sort(exons_int_safe[exons_int_safe['ensembl_trs_id'] == trs].iloc[:, -2:].to_numpy().ravel()))
    
    hits = []
    
    for i in combinations(range(len(cdsrg)), 2):
        if np.array_equal(cdsrg[i[0]], cdsrg[i[1]]):
            hits.append(list(v[list(i)]))
    
    if hits:
        return hits
    else:
        return np.nan

In [75]:
dup('USP9Y')

[['ENST00000651177', 'ENST00000338981']]

In [78]:
all_hits = pd.Series(index=genes, dtype='O')
for x, g in enumerate(genes):
    all_hits[g] = dup(g)
    print(f'Done {x+1} out of {len(genes)}            ', end='\r')

In [80]:
all_hits2 = all_hits[~all_hits.isna()]

In [82]:
all_hits3 = all_hits2.explode()

In [85]:
all_hits3

USP9Y      [ENST00000651177, ENST00000338981]
BPY2       [ENST00000331070, ENST00000382585]
PCDH11Y    [ENST00000333703, ENST00000622698]
BPY2B      [ENST00000382392, ENST00000615850]
TGIF2LY    [ENST00000321217, ENST00000559055]
                          ...                
EIF3I      [ENST00000677760, ENST00000677711]
EIF3I      [ENST00000676801, ENST00000677202]
EIF3I      [ENST00000676801, ENST00000677711]
EIF3I      [ENST00000677202, ENST00000677711]
UBAP2L     [ENST00000428931, ENST00000361546]
Length: 27300, dtype: object

In [91]:
all_hits4.to_frame().to_csv('ens_dup.csv')

In [90]:
all_hits4 = all_hits3.apply(lambda x: x[1]).drop_duplicates()

In [92]:
all_hits4.index.nunique()

5850

In [69]:
v = ens_min[(ens_min['ensembl_gene_name'] == 'TEF') & (ens_min['trs_type'] == 'protein_coding')]['ensembl_trs_id'].values
cdsrg = []

for trs in v:
    cdsrg.append(list(np.sort(exons_int_safe[exons_int_safe['ensembl_trs_id'] == trs].iloc[:, -2:].to_numpy().ravel())))

unique, indices, inverse = np.unique(cdsrg, axis=0, return_index=True, return_inverse=True)

TypeError: The axis argument to unique is not supported for dtype object

In [58]:
exons_int_safe = ens_exons.drop(['5utr_start', '5utr_end', '3utr_start', '3utr_end'], axis=1).dropna(subset=['cds_start', 'cds_end'])
exons_int_safe.loc[:, ['cds_start', 'cds_end']] = exons_int_safe.loc[:, ['cds_start', 'cds_end']].astype(int)

In [59]:
exons_int_safe.iloc[:,-1].to_numpy()

array([     4262,      5511,      7445, ..., 154275483, 154273961,
       154275701])

In [63]:
cdsrg

[[41367533,
  41367599,
  41387351,
  41387668,
  41394096,
  41394316,
  41395745,
  41395960],
 [41382045,
  41382201,
  41387351,
  41387668,
  41394096,
  41394316,
  41395745,
  41395960],
 [41382935, 41382988, 41387351, 41387668, 41394096, 41394274]]

In [64]:
y = [
    [1, 2, 3],
    [1, 2, 3],
    [3, 4, 5],
    [6, 5, 4],
    [4, 2, 5],
    [4, 2, 5],
    [1, 2, 8],
    [1, 2, 3]
]

# Returns unique values of array, indices of that
# array, and the indices that would rebuild the original array
unique, indices, inverse = np.unique(y, axis=0, return_index=True, return_inverse=True)

In [67]:
list(groupby(y)

[([1, 2, 3], <itertools._grouper at 0x13a50b3c640>),
 ([3, 4, 5], <itertools._grouper at 0x13a50b3c910>),
 ([6, 5, 4], <itertools._grouper at 0x13a50b3e3e0>),
 ([4, 2, 5], <itertools._grouper at 0x13a50b3e860>),
 ([1, 2, 8], <itertools._grouper at 0x13a50b3c460>),
 ([1, 2, 3], <itertools._grouper at 0x13a50b3d9f0>)]

In [ ]:
gene = 'UGP2'

v = ens_min[(ens_min['ensembl_gene_name'] == gene) & (ens_min['trs_type'] == 'protein_coding')]['ensembl_trs_id'].values
   
exon_ends = np.sort(ens_exons[ens_exons['ensembl_trs_id'].isin(v)].iloc[:, -2:].to_numpy().ravel())
    
hits = []

cds_ends = gene_sub[['cds_start', 'cds_end']].to_numpy()

cds_unique = np.sort(np.unique(cds_ends.ravel()))

cds_mtx = (cds_unique == cds_ends[..., None]).any(1).astype(int)

seq_mtx = np.array((len(gene_sub), len(cds_unique)))